In [1]:
import sys

In [2]:
!echo $PATH

/home/ec2-user/.conda/envs/kaggle-gw/bin:/opt/conda/condabin:/opt/amazon/openmpi/bin:/opt/amazon/efa/bin:/opt/conda/bin:/usr/local/cuda/bin:/usr/libexec/gcc/x86_64-redhat-linux/7:/opt/aws/bin:/opt/amazon/openmpi/bin:/opt/amazon/efa/bin:/opt/conda/bin:/usr/local/cuda/bin:/usr/libexec/gcc/x86_64-redhat-linux/7:/opt/aws/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin


In [8]:
!pwd

/home/ec2-user/kaggle-gw/notebooks


In [3]:
# Which pythons are we using?
shell_python = !command -v python
print(" shell python: ", shell_python[0])
print("kernel python: ", sys.executable)

 shell python:  /home/ec2-user/.conda/envs/kaggle-gw/bin/python
kernel python:  /home/ec2-user/.conda/envs/kaggle-gw/bin/python


In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal.windows
import scipy.fftpack

from pathlib import Path
from enum import IntEnum, auto
import itertools

from typeguard import typechecked
from typing import List

In [5]:
# Suitable for a 2020ish MacBook Pro
plt.rcParams['figure.dpi']= 140

SMALL_FONT_SIZE = 6
MEDIUM_FONT_SIZE = 8
BIGGER_FONT_SIZE = 10

plt.rc('font', size=SMALL_FONT_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_FONT_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_FONT_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_FONT_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_FONT_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_FONT_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_FONT_SIZE)  # fontsize of the figure title

In [6]:
# What's the best way to set up auto-complete for Kaggle?
# I picked this up from one of the notebooks I was cribbing from.
# %config IPCompleter.use_jedi=False

# import the PyCBC gravitational-wave analysis toolkit
* https://github.com/gwastro/pycbc

Borrowing here from [PyCBC: Making Images](https://www.kaggle.com/alexnitz/pycbc-making-images), by AlexNitz.

In [7]:
import pycbc.types
from pycbc.types import TimeSeries
from pycbc import fft
import pycbc.filter

## Get the data

In [14]:
data_path = Path('/home/ec2-user/subsampled-data')

def list_examples(path, target):
    return [(filename, target) for _, _, filenames in os.walk(path) for filename in filenames]

ones = list_examples(data_path / 'one-target', 1)
zeroes = list_examples(data_path / 'zero-target', 0)

train_df = pd.DataFrame(ones + zeroes, columns=['id', 'target'])
train_df.head()

,id,target
0,00000e74ad.npy,1
1,0000a38978.npy,1
2,0000bb9f3e.npy,1
3,0000d61b7b.npy,1
4,0001016d12.npy,1


### Capture Some Givens

In [15]:
N_SIGNALS = 3
SIGNAL_NAMES = ["LIGO Hanford", "LIGO Livingston", "Virgo"]
SIGNAL_LEN = 4096
SIGNAL_SECONDS = 2.0
DELTA_T = SIGNAL_SECONDS / SIGNAL_LEN
SIGNAL_TIMES = [i * DELTA_T for i in range(SIGNAL_LEN)]

In [ ]:
def signal_path(signal_id, split='train'):
    return str(train_npy_files_path / signal_id[0] / signal_id[1] / signal_id[2] / f"{signal_id}.npy")

In [ ]:
id_to_idx = {row.id: row.Index for row in train_df.itertuples()}

def read_id_signals_and_target(idx):
    _id = train_df['id'][idx]
    signal = np.load(signal_path(_id))
    target = train_df['target'][idx]
    return _id, signal, target

def read_signals_and_target(_id):
    idx = id_to_idx[_id]
    _, signal, target = read_id_signals_and_target(idx)
    return signal, target

## Basic Data Checks

In [ ]:
train_df.head()

In [ ]:
# Any duplicate signal in the data?
train_df["id"].duplicated().sum()

In [ ]:
#  Distribution of the labels
plt.figure(figsize=(1, 1))
sns.countplot(x=train_df["target"], data=train_df)
plt.show()

In [ ]:
## Load a test example that has a strong signal.
test_id = '339f690782'
test_sigs, test_targ = read_signals_and_target(test_id)

# Build some Preprocessing and Graphing Infrastructure

In [ ]:
SIGNAL_COLORS = ['red', 'green', 'blue']

def plot_sig_line(ax, sigs, idx):
    ax.minorticks_on()
    ax.grid(which='major', color='#555555', linestyle='-', linewidth=0.7)
    ax.grid(which='minor', color='#AAAAAA', linestyle=':', linewidth=0.5)
    ax.set_axisbelow(False)

    ax.plot(SIGNAL_TIMES,
           sigs[idx],
           SIGNAL_COLORS[idx])
    
def plot_example_lines(_id, sigs, target):
    fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=[5, 2])
    for i in range(3):
        plot_sig_line(axs[i], sigs, i)
    fig.suptitle(f'id={_id}, target={target}')
    

In [ ]:
for _ in range(4):
    idx = np.random.randint(len(train_df))
    _id, sigs, targ = read_id_signals_and_target(idx)
    plot_example_lines(_id, sigs, targ)         

In [ ]:
@typechecked
def timeseries_from_signal(sig: np.ndarray) -> TimeSeries:
    return TimeSeries(sig, epoch=0, delta_t=DELTA_T)

@typechecked
def timeseries_from_signals(sigs: np.ndarray) -> List[TimeSeries]:
    return [timeseries_from_signal(sigs[i]) for i in range(N_SIGNALS)]

In [ ]:
test_tss = timeseries_from_signals(test_sigs) 

## Preprocess
As baselines, see [the paper reporting the GW150914 discovery](https://iopscience.iop.org/article/10.1088/1361-6382/ab685e) and the corresponding [sample code in PyCBC's docs](https://pycbc.org/pycbc/latest/html/gw150914.html).

Here are the steps we explore:
* Apply a window function (Tukey - tapered cosine window) to reduce [spectral leakage](https://dspillustrations.com/pages/posts/misc/spectral-leakage-zero-padding-and-frequency-resolution.html).
* Whiten the spectrum.
* Apply a bandpass filter.

## Tukey window

Here are two examples of Tukey windows, with different parameters:

In [ ]:
plt.plot(scipy.signal.windows.tukey(4096), label='default alpha');
plt.plot(scipy.signal.windows.tukey(4096, alpha=0.2), label='alpha=0.2')
plt.legend()
plt.show()

Let's define a function to window our data.

In [ ]:
# Given that the most visible signals I have looked at
# (all of the signals?) show up in a t range of roughly (1.3, 1.8),
# we need a shorter, steeper shoulder than the default alpha=0.5.
TUKEY_WINDOW = scipy.signal.tukey(4096, alpha=0.2)

@typechecked
def window(sigs: np.ndarray) -> np.ndarray:
    return sigs * TUKEY_WINDOW

Let's look at our test example before and after windowing.

Before:

In [ ]:
plot_example_lines(test_id, test_sigs, test_targ)

After windowing:

In [ ]:
plot_example_lines(test_id, window(test_sigs), test_targ)

Let's look at the spectrum for one of our test signals:

In [ ]:
test0_fft = scipy.fftpack.fft(test_sigs[0])
# The resulting x axis is given by the fft(...) function.
test0_fft_x = np.linspace(0.0, 1.0/(2.0*DELTA_T), SIGNAL_LEN//2)

fig, ax = plt.subplots()
# The positive frequencies are in the first half of fft(...)'s output.
# The output is complex, so plot its magnitude.
ax.plot(test0_fft_x, np.abs(test0_fft[:SIGNAL_LEN//2]))
ax.set_xlabel('Hz')
ax.set_yscale('log')
plt.show()

Zooming in on the low frequences:

In [ ]:
fig, ax = plt.subplots()
ax.plot(test0_fft_x[:70], np.abs(test0_fft[:70]))
ax.set_xlabel('Hz')
ax.set_yscale('log')
plt.show()

The signal's most intense frequency components (and thus the waves visible on our graphs so far) are at frequencies around 20Hz and down. But from [our reference paper](https://iopscience.iop.org/article/10.1088/1361-6382/ab685e), the GW information is in the 35 to 350Hz range. So we'll need a bandpass filter.

In [ ]:
@typechecked
def bandpass_ts(ts: TimeSeries, 
                lf: float=35.0, 
                hf: float=350.0) -> TimeSeries:
    hp = pycbc.filter.highpass(ts, lf, 8)
    return pycbc.filter.lowpass_fir(hp, hf, 8)
    
@typechecked
def bandpass_sigs(sigs: np.ndarray, 
                  lf: float=35.0, 
                  hf: float=350.0) -> np.ndarray:
    tss = timeseries_from_signals(sigs)
    filtered_tss = [bandpass_ts(ts, lf, hf) for ts in tss]
    return np.stack(filtered_tss)

Here's how our test example looks after bandpass:

In [ ]:
plot_example_lines(test_id, bandpass_sigs(test_sigs), test_targ)

### Define Our Preprocessing Function

Here's a complete preprocessing function modeled closely after the [sample code for GW150914 in PyCBC's docs].

In [ ]:
@typechecked
def preprocess_sig(sig: np.ndarray) -> np.ndarray:
    from pycbc.psd import welch, interpolate
    
    windowed = timeseries_from_signal(window(sig))
    high = pycbc.filter.highpass(windowed, 15, 8)
    
    # whiten

    psd = interpolate(welch(high), 1.0 / high.duration)
    white = (high.to_frequencyseries() / psd ** 0.5).to_timeseries()

    # The above whitening process was taken straight from PyCBC's example code
    # for GW150914, but it adds huge spikes for 0.0 <= t <= 0.1.
    # Rather than sort that out yet (TODO), we tukey out the spike.
    from pycbc.strain import gate_data
    white = gate_data(white, [(0.0, 0.05, 0.05)])
    # Here's an alternative approach from the example notebook we began with.
    # It adds complexity by cropping the time axis.
    # TODO: Is this better or worse?
    # white = high.whiten(0.125, 0.125)

    bandpassed = bandpass_ts(white)

    preprocessed = np.array(bandpassed)
    # Normalize to [0, 1]
    return preprocessed / np.max(np.abs(preprocessed))
        
@typechecked
def preprocess_sigs(sigs: np.ndarray) -> np.ndarray:
    return np.stack([preprocess_sig(sig) for sig in sigs])

In [ ]:
test_psigs = preprocess_sigs(test_sigs)

In [ ]:
plot_example_lines(test_id, test_psigs, test_targ)

## Q-Transform
The Q-Transform is related to the Fourier transform, and very closely related to a wavelet transform. The spectrogram is a possible candidate as input for a CNN model.

### Learn About Q-Transform

## Define Our Q-Transform Function

## Define Our Graphing Functions

In [ ]:
def plot_sig_q(ax, sigs, idx):
    ax.set_yscale('log')
        
    ts = pycbc.types.TimeSeries(sigs[idx, :], epoch=0, delta_t=DELTA_T) 
    # Normalize the noise power at different frequencies.
    ts = ts.whiten(0.125, 0.125)
    qtime, qfreq, qpower = ts.qtransform(DELTA_T, logfsteps=100, qrange=(10, 10), frange=(20, 512))

    ax.pcolormesh(qtime, qfreq, qpower, vmax=15, vmin=0, cmap='viridis', shading='auto')
    
    ax.minorticks_on()
    ax.grid(which='major', color='#DDDDDD', linestyle='-', linewidth=0.7)
    ax.grid(which='minor', color='#CCCCCC', linestyle=':', linewidth=0.5)
    ax.set_axisbelow(False)

In [ ]:
def show_example(sample_id):
    sigs, targ = read_signals_and_target(sample_id)
    
    class Plot(IntEnum):
        SIG0_Q = 0
        SIG1_Q = auto()
        SIG2_Q = auto()
        
        SIG0_LINE = auto()
        SIG1_LINE = auto()
        SIG2_LINE = auto()
        
        ALL_SIGS = auto()
        
    NUM_PLOTS = len(Plot)

    fig, axs = plt.subplots(nrows=NUM_PLOTS, 
                            ncols=1, 
                            sharex=True,
                            figsize=[9, 9])
    
    plot_sig_q(axs[Plot.SIG0_Q], sigs, 0)
    plot_sig_q(axs[Plot.SIG1_Q], sigs, 1)
    plot_sig_q(axs[Plot.SIG2_Q], sigs, 2)
    
    ptss = preprocess_sigs(sigs)

    plot_sig_line(axs[Plot.SIG0_LINE], ptss, 0)
    plot_sig_line(axs[Plot.SIG1_LINE], ptss, 1)
    plot_sig_line(axs[Plot.SIG2_LINE], ptss, 2)
    
    plot_sig_line(axs[Plot.ALL_SIGS], ptss, 0)
    plot_sig_line(axs[Plot.ALL_SIGS], ptss, 1)
    plot_sig_line(axs[Plot.ALL_SIGS], ptss, 2)
    
    fig.suptitle(f'id={sample_id}, target={targ}\n', y=0.1)
    plt.show()

# Show Strong-Signal Examples

In [ ]:
for _id in ['339f690782', '68222c0e9c']:
    show_example(_id)

# Show Random Examples

In [ ]:
for _ in range(100):
    random_idx = np.random.randint(len(train_df))
    show_example(train_df['id'][random_idx])